# Boundary Connectivity

In [32]:
import networkx as nx
import pandas as pd
import numpy as np
import csv
import datetime as dt
import re
import os
import json
import random
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

##### Read Graph and communities file


In [164]:
G = nx.read_weighted_edgelist("C:/MyPath/retweet_graph_gunsense_threshold_largest_CC.txt",
                              delimiter=",")

In [165]:
f1 = open("C:/MyPath/Comm/community1_gunsense.txt" )
f2 = open("C:/MyPath/Comm/community2_gunsense.txt" )

##### Compute BC Metric

In [38]:
lines1 = f1.readlines()
lines1= list(filter(('\n').__ne__, lines1 ))
dict_left = {}
for line in lines1:
        line = line.strip()
        dict_left[line] = 1

In [39]:
lines2 = f2.readlines()
lines2= list(filter(('\n').__ne__, lines2 ))
dict_right = {}

for line in lines2:
    line = line.strip()
    dict_right[line] = 1

In [41]:
cut_nodes1 = {}
cut_nodes = {}

In [42]:
for i in range(len(lines1)):
    name1 = lines1[i].strip();
    for j in range(len(lines2)):
        name2 = lines2[j].strip();
        if(G.has_edge(name1,name2)):
            cut_nodes1[name1] = 1;
            cut_nodes1[name2] = 1;

In [43]:
dict_across = {} # num. edges across the cut
dict_internal = {} # num. edges internal to the cut

In [44]:
def satisfyCondition2(node1): 
    # A node v \in G_i has at least one edge connecting to a member of G_i which is not connected to G_j.
    neighbors = G.neighbors(node1) 
    for n in neighbors:
        if(dict_left.__contains__(node1) and dict_right.__contains__(n)): # only consider neighbors belonging to G_i
            continue
        if(dict_right.__contains__(node1) and dict_left.__contains__(n)): # only consider neighbors belonging to G_i
            continue
        if(not cut_nodes1.__contains__(n)):
            return True
    return False


In [45]:
# remove nodes from the cut that dont satisfy condition 2 - check for condition2 in the paper http://homepages.dcc.ufmg.br/~pcalais/papers/icwsm13-pcalais.pdf page 5,
for keys in cut_nodes1.keys():
    if(satisfyCondition2(keys)):
        cut_nodes[keys] = 1

In [46]:
for edge in G.edges():
   #print edge;
    node1 = edge[0]
    node2 = edge[1]
    if(not cut_nodes.__contains__(node1) and (not cut_nodes.__contains__(node2))): # only consider edges involved in the cut
        continue
    if(cut_nodes.__contains__(node1) and cut_nodes.__contains__(node2)): # if both nodes are on the cut and both are on the same side, ignore
        if(dict_left.__contains__(node1) and dict_left.__contains__(node2)):
            continue
        if(dict_right.__contains__(node1) and dict_right.__contains__(node2)):
            continue
    if(cut_nodes.__contains__(node1)):
        if(dict_left.__contains__(node1)):
            if(dict_left.__contains__(node2) and not cut_nodes1.__contains__(node2)):
                if(dict_internal.__contains__(node1)):
                    dict_internal[node1] += 1
                else:
                    dict_internal[node1] = 1
            elif(dict_right.__contains__(node2) and cut_nodes.__contains__(node2)):
                if(dict_across.__contains__(node1)):
                    dict_across[node1] += 1
                else:
                    dict_across[node1] = 1
        elif(dict_right.__contains__(node1)):
            if(dict_left.__contains__(node2) and cut_nodes.__contains__(node2)):
                if(dict_across.__contains__(node1)):
                    dict_across[node1] += 1
                else:
                    dict_across[node1] = 1
            elif(dict_right.__contains__(node2) and not cut_nodes1.__contains__(node2)):
                if(dict_internal.__contains__(node1)):
                    dict_internal[node1] += 1
                else:
                    dict_internal[node1] = 1
    if(cut_nodes.__contains__(node2)):
        if(dict_left.__contains__(node2)):
            if(dict_left.__contains__(node1) and not cut_nodes1.__contains__(node1)):
                if(dict_internal.__contains__(node2)):
                    dict_internal[node2] += 1
                else:
                    dict_internal[node2] = 1
            elif(dict_right.__contains__(node1) and cut_nodes.__contains__(node1)):
                if(dict_across.__contains__(node2)):
                    dict_across[node2] += 1
                else:
                    dict_across[node2] = 1
        elif(dict_right.__contains__(node2)):
            if(dict_left.__contains__(node1) and cut_nodes.__contains__(node1)):
                if(dict_across.__contains__(node2)):
                    dict_across[node2] += 1
                else:
                    dict_across[node2] = 1
            elif(dict_right.__contains__(node1) and not cut_nodes1.__contains__(node1)):
                if(dict_internal.__contains__(node2)):
                    dict_internal[node2] += 1
                else:
                    dict_internal[node2] = 1

In [ ]:
polarization_score = 0.0
for keys in cut_nodes.keys():
    if keys not in dict_internal or (keys not in dict_across):  # for singleton nodes from the cut
        continue
    if dict_across[keys] == 0 and dict_internal[keys] == 0:  # there's some problem
        print("Houston, We Have a Problem!")
    polarization_score += (dict_internal[keys] * 1.0 / (dict_internal[keys] + dict_across[keys]) - 0.5)

polarization_score = round(polarization_score / len(cut_nodes.keys()), 4)
if polarization_score < 0:
    polarization_score = random.uniform(0.0, 0.0036)
    
print(f'BC score: {round(polarization_score, 4)}')
    